In [1]:
# run only if you have not installed the next libraries
"""
!pip install numpy
!pip install matplotlib
!pip install scipy
!pip install getdist
"""


'\n!pip install numpy\n!pip install matplotlib\n!pip install scipy\n!pip install getdist\n'

### To do:
#### - change acceptance probability
#### - add loss
#### - build neural network
#### - modify leapfrog
#### - function to class MH and HMC

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate as sci
import time
from getdist import plots, MCSamples
import time

%matplotlib inline
dpi = 200

In [2]:
# Este programa implementa Hamiltonian Markov Chain Monte Carlo. El núcleo utilizado se basa en Metropolis Hastings.

class LeapFrog:
    def __init__(self, l, e, dw, m, z, dat, sigma):
        self.l = l
        self.e = e
        self.dw = dw
        self.m = m
        self.x = z
        self.y = dat
        self.sigma = sigma
        self.dim = len(self.m)
        self.nx = network()
        self.nv = network()
        self.nx.zero_grad()
        self.nv.zero_grad()
        
    def solve(self, theta, n, M):        
        qe = theta
        while True:
            vi = np.random.multivariate_normal(mean=np.zeros(self.dim), cov=np.diag(self.m))
            ve = vi
            self.H = []
            self.X = []
            self.V = []
            self.X.append(theta)
            self.V.append(ve)
            self.nx.forward(torch.FloatTensor(np.concatenate((qe, ve, [np.cos(2*np.pi*n/M), np.sin(2*np.pi*n/M)]))))
            self.nv.forward(torch.FloatTensor(np.concatenate((qe, ve, [np.cos(2*np.pi*n/M), np.sin(2*np.pi*n/M)]))))
            self.qx, self.sx, self.tx = self.nx.get()
            self.qv, self.sv, self.tv = self.nv.get()
            
            self.qx = self.qx.detach.numpy()
            self.sx = self.sx.detach.numpy()
            self.tx = self.tx.detach.numpy()
            self.qv = self.qv.detach.numpy()
            self.sv = self.sv.detach.numpy()
            self.tv = self.tv.detach.numpy()
            for i in range(self.l):
                v = ve*np.exp(0.5*self.e*self.sv) - 0.5*self.e*(gradiente(self.dw, qe, self.x, self.y, self.sigma)*
                                                                 np.exp(self.e*self.qv) + self.tv) # actualiza momento en e/2
                qe = qe*np.exp(self.e*self.sx) + self.e*(ve*np.exp(self.e*self.qx) + self.tx)
                if qe[0]<0:
                    qe[0] = qe[0]*-1
                    ve[0] = ve[0]*-1
                if qe[1]<0:
                    qe[1] = qe[1]*-1
                    ve[1] = ve[1]*-1
                if qe[2]>1/3:
                    qe[2] = 1/3 - qe[2]
                    ve[2] = ve[2]*-1
                ve = ve*np.exp(0.5*self.e*self.sv) - 0.5*self.e*(gradiente(self.dw, qe, self.x, self.y, self.sigma)*
                                                                 np.exp(self.e*self.qv) + self.tv)
                self.V.append(ve)
                self.X.append(qe)
                if i + 1==self.l: 
                    i += 1
                self.H.append(hamiltoniano(p=ve*self.m, dat=self.y, sigma=self.sigma, theta=theta, z=self.x, m=self.m))
            if i==self.l: 
                break
    
    def get(self):
        return np.array(self.X), np.array(self.V*self.m), np.array(self.H)

def likelihood(mod, dat, sigma): # retorna escalar, log(L)
    sig = np.diagonal(sigma)
    L = -0.5*chi2(mod, dat, sigma)[0]  + np.sum(-0.5*np.log(2*np.pi*sig**2))
    #pp = np.argwhere((a1==-np.inf))
    #a1[pp] = 0
    return L


def chi2(mod, dat, sigma):
    sig = np.diagonal(sigma)
    AA = np.sum(((dat - mod)/sig)**2)
    BB = np.sum((dat - mod)/sig**2)	
    CC = np.sum(1/sig**2)
    chi = AA - (BB**2)/CC
    return chi, BB/CC


def prior(theta): # log(pi)
    ct = 1
    r = np.diag(np.ones(len(theta))*ct)
    p = -0.5*np.log(np.linalg.det(2*np.pi*r)) - 0.5*theta.dot((np.linalg.inv(r)).dot(theta)) 
    return p


def acepta_hmc(ec, ep, EC, EP, x, X):
    alpha = min(- EP - EC + ep + ec, 0) # log(alpha)
    u = np.log(np.random.uniform())
    if u<alpha:
        return X, EP, np.exp(alpha)
    else:
        return x, ep, np.exp(alpha)
    
    
def acepta_mh(T1, pos1, T2, pos2, m1, m2):
    alpha = min(pos2 - pos1, 0) # log(alpha)
    u = np.log(np.random.uniform())
    if u<alpha:
        return T2, pos2, m2
    else:
        return T1, pos1, m1    


def EHubble(theta, z): # parametro de hubble
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]
    arg = om0*(1 + z)**3 + (1 - om0 - ol)*(1 + z)**2 + ol*(1 + z)**(3*(1 + w))
    EE = np.sqrt(arg)
    return EE, arg


def modelo(theta,z): # modulo de la distancia teorico
    om0 = theta[0]
    ol = theta[1]
    omega_k = 1 - om0 - ol
    E = EHubble(theta, z)[0]
    I = sci.cumtrapz(1/(E + 1e-300), z, initial=0)+z[0]*((1/(E + 1e-300))[0] + 1)/2 # estabilidad numerica
    o_k_s = np.sqrt(abs(omega_k))
    if omega_k==0:
        dl = (1 + z)*I
    elif omega_k<0:
        dl = (1 + z)*np.sin(o_k_s*I)/(o_k_s + 1e-300) # estabilidad numerica
    elif omega_k>0:	
        dl = (1 + z)*np.sinh(o_k_s*I)/(o_k_s + 1e-300) # estabilidad numerica
    dist = 5*np.log10(dl + 1e-300) # estabilidad numerica
    #f (-np.inf==dist).any(): 
    #    print(theta)
    return dist


def tasa(tant, tpos):
    l = len(tant)
    if np.sum(tant==tpos)==l:
        c = 0
    else:
        c = 1
    return c


def revisa(theta, z):
    arg = EHubble(theta, z)[1]
    bol = np.sum(arg<0)
    print(bol)
    if bol>0:
        a = 0 # raiz imaginaria 
    else:
        a = 1 # raiz real
    return a


def argmin2(t1, t2, t3, V): # busca en base a vector chi2 y devulve minimos de los parametros
    amin = np.argmin(V)
    return t1[amin], t2[amin], t3[amin]


def revisa1(X):
    x = X[0]
    y = X[1]
    z = X[2]
    xlim = np.array([0, 1])
    ylim = xlim
    zlim = np.array([-np.inf, 1/3])
    if xlim[0]<x<xlim[1] and ylim[0]<y<ylim[1] and zlim[0]<z<zlim[1]:
        return 1
    else:
        return 0


def potencial(dat, sigma, theta, z):
    mod = modelo(theta, z)
    print(mod)
    u = - likelihood(mod, dat, sigma) #- prior(theta) 
    return u


def cinetica(p, m):
    k = np.sum(p**2/2/m)
    return k


def hamiltoniano(p, dat, sigma, theta, z, m=1):
    h = cinetica(p, m) + potencial(dat, sigma, theta, z)
    return h


def gradiente(dw, theta, z, dat, sigma):
    tf = theta + dw
    tb = theta - dw
    grad = (potencial(dat, sigma, tf, z) - potencial(dat, sigma, tb, z))/(2*dw)
    return grad


def HMC(modelo, datos, ds, dg, N, L, params, q0, cov_mod, m, M, des=0.24):
    """
    datos: X, F(X)
    N: numero de iteraciones o epocas
    M: tamaño del minibatch
    params: ['p1', 'p2', ..., 'pn']
    cov: matriz de covarianza de datos
    """
    # Matrices de datos de la cadena
    X = datos[0]
    Y = datos[1]
    chain = [] 
    post = [] 
    chi_2 = []
    Ratio = []
    acept = 0
    mod1 = modelo(q0, X)
    pos1 = potencial(Y, cov_mod, q0, X)
    Chi1 = chi2(mod1, Y, cov_mod)[0]
    chain.append(q0)
    post.append(pos1)
    chi_2.append(Chi1)
    Ratio.append(100)
    
    leap = LeapFrog(l=L, e=ds, dw=dg, m=m, z=X, dat=Y, sigma=cov_mod)

    Ti = time.time()
    for i in range(N):
        qo = np.random.uniform(low=[0,0,-5], high=[1, 1, 1/3], size=(M, 3))
        Loss = 0
        for j in range(M):
            q = chain[j]
            while True:
                leap.solve(q, i, M)
                Q, P, H = leap.get()
                Q1 = Q[-1]
                P1 = P[-1]
                if revisa1(Q1):
                    t = cinetica(P1[0], m)
                    u = potencial(Y, cov_mod, q, X)
                    T = cinetica(P1, m)
                    U = potencial(Y, cov_mod, Q1, X)
                    A = acepta_hmc(t, u, T, U, q, Q1)
                    break
            while True:
                leap.solve(qo[j], i, M)
                Qo, Po, Ho = leap.get()
                Qo1 = Qo[-1]
                Po1 = Po[-1]
                if revisa1(Qo1):
                    to = cinetica(Po1[0], m)
                    uo = potencial(Y, cov_mod, qo[j], X)
                    To = cinetica(Po1, m)
                    Uo = potencial(Y, cov_mod, Qo1, X)
                    Ao = acepta_hmc(to, uo, To, Uo, qo[j], Qo1)
                    break
            Loss += loss(q, Q1, qo[j], Qo1, A[2], Ao[2], par1=0.1, par2=0.1)
            chain.append(A[0])
            post.append(A[1])
            mod1 = modelo(A[0], X)
            Chi1 = chi2(mod1, Y, cov_mod)[0]
            chi_2.append(Chi1)
            # ratio de aceptacion
            acept += tasa(chain[i], chain[i + 1]) 
            Ratio.append(acept/(M*i + j + 1)*100)                               
        optimizer.zero_grad()
        # forward + backward + optimize
        Loss.backward()
        optimizer.step()
        # print statistics
        print('Loss', Loss/M)         
        if i%100==0:
            print(i)
            print('ratio', Ratio[i])

    Tf = time.time()
    print('Tiempo cadena', np.around(Tf - Ti, 0), 's')

    post = np.array(post)
    chain = np.array(chain)
    chi_2 = np.array(chi_2)
    Ratio = np.array(Ratio)
    H = np.array(H)

    t1 = chain[:,0]
    t2 = chain[:,1]
    t3 = chain[:,2]

    # busca argumento del minimo de chi2
    t1m, t2m, t3m = np.around(argmin2(t1, t2, t3, chi_2),3)
    mins = [t1m, t2m, t3m]
    muestras = {}
    for i in range(len(params)):
        muestras[params[i]] = chain[:, i]
    return muestras, Ratio, chi_2, post, mins


def MH(modelo, datos, N, params, q0, cov_mod, cov_prop, des=0.24):
    """
    datos: X, F(X)
    params: ['p1', 'p2', ..., 'pn']
    cov: matriz de covarianza de datos
    """
    # Matrices de datos de la cadena
    #pid = PID(kp=10, ki=10, kd=10, o_min=2, o_max=20)
    T0 = q0
    X = datos[0]
    Y = datos[1]
    chain = [] 
    post = [] 
    chi_2 = []
    Ratio = []
    mod = []
    acept = 0
    mod0 = modelo(T0, X)
    chi0 = chi2(mod0, Y, cov_mod)[0]
    pos0 = likelihood(mod0, Y, cov_mod) + prior(T0)
    mod.append(mod0)
    chain.append(T0)
    post.append(pos0)
    chi_2.append(chi0)
    Ratio.append(100)

    # pasos de cadena
    Ti = time.time()
    for i in range(N):
        # revisa si se paso umbral de burn in
        """	
        if chi_2[i]<=580 and d==0 and o!=0:
            covarianza = COV[o]
            d = 1
            print('actualizada')
            print(covarianza)
        """	
        # selecciona ultimo elemento de la cadena
        T0 = chain[i]
        # itera hasta que encuentra un proposal valido
        while True:
            T1 = np.random.multivariate_normal(mean=T0, cov=cov_prop)
            if revisa1(T1):
                break
        # selecciona ultimo modelo
        mod0 = mod[i]
        # calcula modelo con proposal
        mod1 = modelo(T1, X)
        # selecciona ultima dis. post.
        pos0 = post[i]
        # calcula nueva dist. post.
        pos1 = likelihood(mod1, Y, cov_mod) + prior(T1)
        # decision de aceptacion
        A = acepta_mh(T0, pos0, T1, pos1, mod1, mod1)
        # guarda la variable aceptada (puede ser la anterior o proposal)
        chain.append(A[0])
        post.append(A[1])
        mod.append(A[2])
        chi_2.append(chi2(A[2], Y, cov)[0])
        # ratio de aceptacion
        acept += tasa(chain[i], chain[i + 1]) 
        Ratio.append(acept/(i+1)*100)
        if i%100==0:
            print(i)
            print('ratio', Ratio[i])

    Tf = time.time()
    print('Tiempo cadena', np.around(Tf - Ti, 0), 's')
    
    ratio = acept/N*100
    print('ratio %', np.rint(ratio))

    post = np.array(post)
    chain = np.array(chain)
    chi_2 = np.array(chi_2)
    Ratio = np.array(Ratio)
  
    t1 = chain[:,0]
    t2 = chain[:,1]
    t3 = chain[:,2]

    # busca argumento del minimo de chi2
    t1m, t2m, t3m = np.around(argmin2(t1, t2, t3, chi_2),3)
    mins = [t1m, t2m, t3m]
    muestras = {}
    for i in range(len(params)):
        muestras[params[i]] = chain[:, i]
    return muestras, Ratio, chi_2, post, mins


def plot(arr1, arr2, keys, names, save=None):
    fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
    ax1.scatter(arr1[0][keys[0]], arr1[0][keys[1]], marker='.', alpha=0.1)
    ax1.scatter(arr1[0][keys[0]][0], arr1[0][keys[1]][0], marker='x', color='red', alpha=1, label='inicio')
    ax1.scatter(arr1[4][0], arr1[4][1], marker='o', color='black', alpha=1, label='best fit')
    ax2.scatter(arr2[0][keys[0]], arr2[0][keys[1]], marker='.', alpha=0.1)
    ax2.scatter(arr2[0][keys[0]][0], arr2[0][keys[1]][0], marker='x', color='red', alpha=1, label='inicio')
    ax2.scatter(arr2[4][0], arr2[4][1], marker='o', color='black', alpha=1, label='best fit')
    ax1.set_title('HMC')
    ax2.set_title('MH')
    ax1.legend()
    ax2.legend()
    ax2.set_xlabel(names[0])
    ax2.set_ylabel(names[1])
    ax1.set_ylabel(names[1])
    if save!=None:
        fig.savefig('muestras_'+keys[0]+'_'+keys[1], dpi=dpi)
    return


def autocorrelation(x, kmax=None):
    N = len(x)
    xmean = np.mean(x)
    den = np.sum((x - xmean)*(x - xmean))    
    if kmax==None:
        kmax = int(N/2)
    C = []
    for i in range(kmax):
        if i==0:
            corr = 1
        else:
            corr = np.sum((x[i:] - xmean)*(x[:-i] - xmean))/den
        C.append(corr)
    return np.array(C)


In [3]:
# saved things directory
direc = '/home/mauricio/Documents/Uni/Intro_2/' + 'gal.txt'

# Carga de datos
redshift = np.genfromtxt('gal.txt', usecols=(1))
mu_obs = np.genfromtxt('gal.txt', usecols=(2)) # m - M
cov = np.genfromtxt('gal.txt', usecols=(3))

p = np.argsort(redshift)
redshift = redshift[p]
mu_obs = mu_obs[p]
cov = cov[p]
cov = np.diag(cov)

In [4]:
# configuracion cadena

# params
labs = [r'$\Omega_{m}$', r'$\Omega_{\Lambda}$', r'w']
labs1 = [r'\Omega_{m}', r'\Omega_{\Lambda}', r'w']

M = -19.3182761161

N = 200

## Metroplis-Hastings

In [ ]:
# covarianza inicial
cov_ini = np.diag(np.array([0.4, 1.125, 4])**2)*0.5e-2
print(np.diag(cov_ini))
q0 = np.random.uniform(low=[0,0,-5], high=[1, 1, 1/3], size=3)
print('q0', q0)
R_mh = MH(modelo, [redshift, mu_obs], N=N, params=['om', 'ol', 'w'], q0=q0, cov_mod=cov, cov_prop=cov_ini)

## Hamiltonian Monte Carlo
##### Select L between 5 and 10 steps

In [ ]:
ds = 1e-2
m = np.array([1, 1, 1])
print('q0', q0)
R_hmc = HMC(modelo, [redshift, mu_obs], ds=ds, dg=1e-6, N=N, L=8, params=['om', 'ol', 'w'], 
            q0=q0, cov_mod=cov, m=m, M=2)

## Resultados

In [ ]:
names = [r'$\Omega_{m}$', r'$\Omega_{\Lambda}$']
keys = ['om', 'ol']
plot(R_hmc, R_mh, keys, names, save='yes')

In [ ]:
names = [r'$\Omega_{m}$', r'$\omega$']
keys = ['om', 'w']
plot(R_hmc, R_mh, keys, names, save='yes')

In [ ]:
names = [r'$\Omega_{\Lambda}$', r'$\omega$']
keys = ['ol', 'w']
plot(R_hmc, R_mh, keys, names, save='yes please')

In [ ]:
ndim = 3
names = ["x%s"%i for i in range(ndim)]
labels = ["x_%s"%i for i in range(ndim)]
t1 = R_hmc[0]['om']
t2 = R_hmc[0]['ol']
t3 = R_hmc[0]['w']
samps = np.vstack((t1, t2, t3)).T
samples = MCSamples(samples=samps, names=labs1, labels=labs1)

#Triangle plot
g = plots.getSubplotPlotter()
samples.updateSettings({'contours': [0.68, 0.95, 0.99]})
g.settings.num_plot_contours = 3
g.triangle_plot([samples], filled=True)


In [ ]:
ac_hmc_om = autocorrelation(R_hmc[0]['om'])
ac_mh_om = autocorrelation(R_mh[0]['om'])
   
plt.plot(ac_hmc_om, label='HMC')
plt.plot(ac_mh_om, label='MH')
plt.title('autocorrelation')
plt.legend()
plt.xlabel('k')

In [10]:
x = np.array([0.5, 0.5, -0.5])
pot = potencial(mu_obs, cov, x, redshift)
print(pot)

[-9.10535665e+00 -9.10535665e+00 -9.10535665e+00 -9.10142032e+00
 -9.09083573e+00 -9.08130418e+00 -9.07641006e+00 -9.07641006e+00
 -9.06207838e+00 -9.04269347e+00 -8.96428855e+00 -8.96428855e+00
 -8.92367322e+00 -8.92085801e+00 -8.91755929e+00 -8.90364314e+00
 -8.88920143e+00 -8.86673430e+00 -8.86503561e+00 -8.83287826e+00
 -8.80957405e+00 -8.80270677e+00 -8.79345713e+00 -8.79345713e+00
 -8.75522596e+00 -8.71504813e+00 -8.66868867e+00 -8.62318792e+00
 -8.59990255e+00 -8.56542891e+00 -8.56542891e+00 -8.55814373e+00
 -8.53148557e+00 -8.52039785e+00 -8.43539436e+00 -8.42196848e+00
 -8.39014505e+00 -8.37963830e+00 -8.35877348e+00 -8.34841448e+00
 -8.31762564e+00 -8.28796352e+00 -8.27722971e+00 -8.26923774e+00
 -8.25730514e+00 -8.25730514e+00 -8.22775085e+00 -8.17935445e+00
 -8.17935445e+00 -8.17255118e+00 -8.16980089e+00 -8.15006062e+00
 -8.14138518e+00 -8.14138518e+00 -8.11928501e+00 -8.08562498e+00
 -8.08076633e+00 -8.07646660e+00 -8.05959565e+00 -8.05826266e+00
 -8.04795295e+00 -8.02899

In [ ]:
X = torch.FloatTensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]))
Y = torch.LongTensor(np.array([0, 1, 1, 0]))

C = 2
dataset = BatchingDataset(X, Y, C=C)

print('X', dataset.datos)
print('clases', dataset.clases)

In [ ]:
net = Net()
net.zero_grad()
print(net)

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [ ]:
train(net, dataset, optimizer, epochs=1000, minibatches=2)

In [ ]:
print(net(X))

In [ ]:
loss(1, 2, 0.1, 0.3, 0.9, 0.2, 1e-1, 1e-1)

In [16]:
def EHubble(theta, z): # parametro de hubble
    om0 = theta[0]
    ol = theta[1]
    w = theta[2]
    arg = om0*(1 + z)**3 + (1 - om0 - ol)*(1 + z)**2 + ol*(1 + z)**(3*(1 + w))
    #print(arg)
    EE = np.sqrt(arg)
    return EE, arg


def modelo(theta,z): # modulo de la distancia teorico
    om0 = theta[0]
    ol = theta[1]
    omega_k = 1 - om0 - ol
    E = EHubble(theta, z)[0]
    #print(E)
    I = sci.cumtrapz(1/(E + 1e-300), z, initial=0)+z[0]*((1/(E + 1e-300))[0] + 1)/2 # estabilidad numerica
    #print(I)
    o_k_s = np.sqrt(abs(omega_k))
    #print(o_k_s)
    if omega_k==0:
        dl = (1 + z)*I
    elif omega_k<0:
        dl = (1 + z)*np.sin(o_k_s*I)/(o_k_s + 1e-300) # estabilidad numerica
    elif omega_k>0:	
        dl = (1 + z)*np.sinh(o_k_s*I)/(o_k_s + 1e-300) # estabilidad numerica
    print(dl)
    dist = 5*np.log10(dl + 1e-300) # estabilidad numerica
    #f (-np.inf==dist).any(): 
    #    print(theta)
    return dist

In [17]:
x = np.array([0.5, 0.4, -0.5])
modelo(x, redshift)

[0.01509561 0.01509561 0.01509561 0.015123   0.01519688 0.01526372
 0.01529815 0.01529815 0.01539944 0.0155375  0.01610865 0.01610865
 0.01641273 0.01643402 0.016459   0.01656479 0.01667531 0.0168487
 0.01686188 0.0171134  0.01729801 0.01735279 0.01742685 0.01742685
 0.01773632 0.01806748 0.01845727 0.01884802 0.01905118 0.01935597
 0.01935597 0.01942101 0.01966084 0.01976147 0.0205502  0.02067762
 0.02098282 0.02108456 0.02128809 0.02138986 0.02169523 0.02199354
 0.0221025  0.02218398 0.02230619 0.02230619 0.02261178 0.02312126
 0.02312126 0.02319379 0.02322318 0.02343521 0.023529   0.023529
 0.02376962 0.02414085 0.02419491 0.02424285 0.02443188 0.02444688
 0.0245632  0.02477851 0.02495711 0.02505918 0.02516126 0.02536544
 0.0257739  0.02587603 0.02618249 0.02628466 0.02632348 0.02638683
 0.02678438 0.02689784 0.0271117  0.02765655 0.02781817 0.02788847
 0.02802277 0.0281916  0.02863679 0.02873509 0.02882925 0.02955837
 0.03017532 0.03033199 0.03068565 0.03092046 0.03099736 0.0311983

array([-9.10574620e+00, -9.10574620e+00, -9.10574620e+00, -9.10181057e+00,
       -9.09122785e+00, -9.08169799e+00, -9.07680474e+00, -9.07680474e+00,
       -9.06247563e+00, -9.04309420e+00, -8.96470369e+00, -8.96470369e+00,
       -8.92409602e+00, -8.92128135e+00, -8.91798325e+00, -8.90406976e+00,
       -8.88963082e+00, -8.86716804e+00, -8.86546968e+00, -8.83331863e+00,
       -8.81001904e+00, -8.80315313e+00, -8.79390533e+00, -8.79390533e+00,
       -8.75568188e+00, -8.71551230e+00, -8.66916252e+00, -8.62367147e+00,
       -8.60039112e+00, -8.56592501e+00, -8.56592501e+00, -8.55864143e+00,
       -8.53198918e+00, -8.52090395e+00, -8.43591984e+00, -8.42249708e+00,
       -8.39068112e+00, -8.38017685e+00, -8.35931700e+00, -8.34896049e+00,
       -8.31817908e+00, -8.28852422e+00, -8.27779306e+00, -8.26980307e+00,
       -8.25787344e+00, -8.25787344e+00, -8.22832655e+00, -8.17994247e+00,
       -8.17994247e+00, -8.17314096e+00, -8.17039138e+00, -8.15065621e+00,
       -8.14198304e+00, -